In [3]:

def get_gesture_counts(participant):
	gestures = {"drag":0,
				"draw":0,
				"ui":0,
				"tap":0,
				"doubletap":0,
				"tripletap":0,
				"hold":0,
				"pinch":0,
				"rev_pinch":0,
				"lasso":0,
				"box":0,
				"voice":0,
				"other":0}

	#Get all the events
	events = []
	for task in participant["tasks"].keys():
		events.extend(participant["tasks"][task])

	for event in events:
	 	if event["event_type"] == "tap":
	 		#Taps need special handling, as they might be double, triple, or hold
	 		if event["hold"]:
	 			gestures["hold"] += 1
	 		elif event["count"] == 2:
	 			gestures["doubletap"] += 1
	 		elif event["count"] == 3:
	 			gestures["tripletap"] += 1
	 		else:
	 			gestures["tap"] += 1
		elif event["event_type"] == "drag":
			#Drags might be drag or might be draw
			if event["draw"] is None:
				gestures["drag"] += 1
			else:
				gestures["draw"] += 1
		elif event["event_type"] == "pinch":
			#pinch can be pinch or reverse
			if event["reverse"]:
				gestures["rev_pinch"] += 1
			else:
				gestures["pinch"] += 1
		elif event["event_type"] == "voice_command":
			gestures["voice"] += 1
		elif event["event_type"] == "ui":
			gestures["ui"] += 1
		elif event["event_type"] == "memo":
			#Don't do anything with memos
			pass
		elif event["event_type"] == "lasso":
			gestures["lasso"] += 1
		elif event["event_type"] == "box_select":
			gestures["box"] += 1
		elif event["event_type"] == "other":
			gestures["other"] += 1		
		else:
			#This is an error, some event type wasn't handled
			print event["event_type"]

	return gestures

In [4]:
def average_counts(counts):
	participant_count = len(counts.keys())
	totals = {}
	#Collect the total counts
	for p in counts.keys():
		p_count = counts[p]
		for gesture in p_count.keys():
			if gesture in totals.keys():
				totals[gesture] += p_count[gesture]
			else:
				totals[gesture] = p_count[gesture]
	#Average across participants
	for gesture in totals.keys():
		totals[gesture] = totals[gesture]/float(participant_count)

	return totals

In [5]:
import all_data_handler
import pandas

In [6]:
adh = all_data_handler.UserData()

What I actually want to do here is have a dataframe where each user is also tagged with what condition they were in, so I can run ANOVA on that, with the condition as my categorical variable. 

In [92]:
data = []
for condition in adh.conditionMap.keys():
    counts = adh.applyCondition(get_gesture_counts, condition)
    #Convert to a list of dicts with user as a parameter of the dictionary
    #First put the user ID and the condition in the data
    for entry in counts:
        counts[entry]["user"] = entry
        counts[entry]["condition"] = condition
        #Tag as a multi-robot or more-or-less single robot condition
        if condition == "one" or condition == "unknown":
            counts[entry]["multi"] = False
        else:
            counts[entry]["multi"] = True
    data.extend(counts.values())

In [93]:
df = pandas.DataFrame(data)

In [94]:
df.set_index("user")

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0,unknown,0,19,3,1,0,False,1,1,0,0,0,0,2
26,0,unknown,5,25,4,2,0,False,1,0,4,4,1,0,0
21,0,unknown,0,13,3,2,1,False,1,1,0,2,0,0,0
16,0,unknown,0,34,1,0,0,False,0,0,0,0,0,0,0
1,0,unknown,7,28,5,5,0,False,0,1,2,2,0,0,0
6,0,unknown,0,20,18,0,0,False,2,1,3,32,0,0,0
25,0,thousand,0,17,5,0,0,True,0,1,0,0,0,0,8
15,0,thousand,0,36,21,0,10,True,0,0,2,0,0,0,2
10,0,thousand,0,18,1,0,0,True,21,0,0,1,0,0,1


So that gets my data into a nice frame, now how do I tell jupyter to do ANOVA to it?

In [52]:
import statsmodels
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [95]:
model = ols('drag ~ multi', data=df).fit()

In [96]:
table = sm.stats.anova_lm(model, typ=1)

In [97]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,2376.200000,2376.200000,4.12045,0.051967
Residual,28,16147.166667,576.684524,NaN,NaN


I have two problems here. The first is that I'm not sure that I'm expressing the dependence between the condition and the variable correctly, and the second is that I don't know how to interpret the output. I think that a low PR(>F) is a good thing, but I'm not sure how low is good enough to say that a given gesture is related to the condition. 

In [98]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   drag   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     4.120
Date:                Mon, 23 Apr 2018   Prob (F-statistic):             0.0520
Time:                        20:32:14   Log-Likelihood:                -136.89
No. Observations:                  30   AIC:                             277.8
Df Residuals:                      28   BIC:                             280.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------
Intercept        17.6667      6.932      2.548

I think that what this means is that I can accept that drag is conditional on whether it's a multirobot condition with 1-0.052 = 0.948 likelyhood

In [115]:
df = pandas.DataFrame(data)

In [116]:
df.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0,unknown,0,19,3,1,0,False,1,1,0,0,0,0,2
26,0,unknown,5,25,4,2,0,False,1,0,4,4,1,0,0
21,0,unknown,0,13,3,2,1,False,1,1,0,2,0,0,0
16,0,unknown,0,34,1,0,0,False,0,0,0,0,0,0,0
1,0,unknown,7,28,5,5,0,False,0,1,2,2,0,0,0
6,0,unknown,0,20,18,0,0,False,2,1,3,32,0,0,0
25,0,thousand,0,17,5,0,0,True,0,1,0,0,0,0,8
15,0,thousand,0,36,21,0,10,True,0,0,2,0,0,0,2
10,0,thousand,0,18,1,0,0,True,21,0,0,1,0,0,1


In [119]:
model = ols('lasso ~ multi', data=df).fit()

In [120]:
table = sm.stats.anova_lm(model, typ=1)

In [121]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,341.688889,341.688889,4.117127,0.052055
Residual,28,2323.777778,82.992063,NaN,NaN


In [122]:
model = ols('lasso ~ condition', data=df).fit()

In [123]:
table = sm.stats.anova_lm(model, typ=1)

In [124]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,592.800000,148.200000,1.787552,0.162909
Residual,25,2072.666667,82.906667,NaN,NaN


This makes it seem like whether the condition is a multi-robot condition is a stronger predictor (0.052 vs 0.163) of the use of lasso than what the actual condition is.

In [105]:
df.corr(method='pearson')

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
box,1.000000,-0.079315,-0.204529,-0.078915,0.421662,-0.046383,0.275703,-0.117489,-0.054159,-0.075523,0.199250,-0.094381,0.730521,-0.042286
doubletap,-0.079315,1.000000,0.268415,-0.143215,0.422243,0.054494,-0.070469,-0.167361,0.187928,0.101987,0.025298,0.298255,-0.027499,-0.147862
drag,-0.204529,0.268415,1.000000,0.073841,0.043100,0.112012,0.358164,0.001384,0.248301,-0.154419,0.322378,0.212972,-0.232808,-0.229847
draw,-0.078915,-0.143215,0.073841,1.000000,-0.098299,0.561740,0.134551,-0.139085,-0.089239,0.120977,-0.019500,0.058253,-0.143913,-0.088714
hold,0.421662,0.422243,0.043100,-0.098299,1.000000,0.085924,0.040989,-0.250393,-0.188132,-0.040033,0.356689,0.314309,0.326193,-0.233795
lasso,-0.046383,0.054494,0.112012,0.561740,0.085924,1.000000,0.358038,-0.088309,0.341111,-0.108431,-0.070712,0.508055,-0.104306,-0.132712
multi,0.275703,-0.070469,0.358164,0.134551,0.040989,0.358038,1.000000,0.299581,0.155211,-0.048834,0.194060,0.171163,0.166127,0.180548
other,-0.117489,-0.167361,0.001384,-0.139085,-0.250393,-0.088309,0.299581,1.000000,-0.029667,0.508409,-0.061970,-0.047807,-0.031288,-0.112941
pinch,-0.054159,0.187928,0.248301,-0.089239,-0.188132,0.341111,0.155211,-0.029667,1.000000,-0.102324,0.094522,-0.091085,-0.091702,-0.068383
rev_pinch,-0.075523,0.101987,-0.154419,0.120977,-0.040033,-0.108431,-0.048834,0.508409,-0.102324,1.000000,-0.062783,-0.055724,-0.005496,-0.144668


Correlation shows that whether the condition is a multirobot condition correlates most strongly with box, drag, other, and lasso. I am not sure why drag and other got in there, although drag is also the most common gesture by far. I suspect that because there were a few multi-robot conditions that had very high counts of "other", but no single-robot conditions that have high counts of "other", the correlation isn't really representative of people's choices so much as it is a couple of outliers. 

Box and UI got a really strong correlation (0.73), which is probably because of box select and menu interactions being the main interaction method of RTS games. Adding whether a person plays RTS games to the data set would help confirm this, as box, ui, and RTS should all be highly correlated if that is the case. 

In [138]:
model = ols('ui ~ box', data=df).fit()

In [139]:
table = sm.stats.anova_lm(model, typ=1)

In [140]:
table

,df,sum_sq,mean_sq,F,PR(>F)
box,1,1651.787908,1651.787908,32.042174,0.000005
Residual,28,1443.412092,51.550432,NaN,NaN


This makes intuitive sense to me, as the use of box and UI are highly correlated, so one can predict the other.

In [143]:
model = ols('box ~ ui', data=df).fit()
table = sm.stats.anova_lm(model, typ=1)

In [144]:
table

,df,sum_sq,mean_sq,F,PR(>F)
ui,1,639.041406,639.041406,32.042174,0.000005
Residual,28,558.425261,19.943759,NaN,NaN


Should switching those have changed the PR(>F) value? I don't suppose it should, correlation goes both ways. 

In [145]:
import scipy.stats as stats

Scipy stats has f_oneway, so I can calculate the averages of each group, and then do that to see if the groups have the same population mean, although the means of means is probably not really what I want there. I could also get the totals for each group, but that also seems like a bad plan. 

In [150]:
all_counts = {}
for condition in adh.conditionMap.keys():
    counts = adh.applyCondition(get_gesture_counts, condition)
    avg = average_counts(counts)
    all_counts[condition] = avg

In [159]:
as_lists = {}
for condition in adh.conditionMap.keys():
    as_lists[condition] = [all_counts[condition][x] for x in sorted(all_counts[condition].keys())]

In [161]:
stats.f_oneway(as_lists['unknown'], as_lists['one'], as_lists['ten'], as_lists['hundred'], as_lists['thousand'])

F_onewayResult(statistic=1.4784337089500879, pvalue=0.22002503502915216)

That's not great, I want a very low P (e.g. less than 0.05). Check to see if unknown and one have different pop means (they shouldn't...)

In [162]:
stats.f_oneway(as_lists['unknown'], as_lists['one'])

F_onewayResult(statistic=0.55200384002671332, pvalue=0.46471106985119826)

In [166]:
for x in as_lists.keys():
    for y in as_lists.keys():
        if x != y:
            print x, y, stats.f_oneway(as_lists[x], as_lists[y])

unknown thousand F_onewayResult(statistic=0.0021886962395856746, pvalue=0.96307289865537826)
unknown hundred F_onewayResult(statistic=0.81132768286727719, pvalue=0.37667821218931918)
unknown ten F_onewayResult(statistic=1.7036814881039262, pvalue=0.20418288297121795)
unknown one F_onewayResult(statistic=0.55200384002671332, pvalue=0.46471106985119826)
thousand unknown F_onewayResult(statistic=0.002188696239585675, pvalue=0.96307289865537826)
thousand hundred F_onewayResult(statistic=0.74646064762372843, pvalue=0.39615212673610622)
thousand ten F_onewayResult(statistic=1.6498127555285791, pvalue=0.21124481628590228)
thousand one F_onewayResult(statistic=0.6589196563127182, pvalue=0.42491972670585898)
hundred unknown F_onewayResult(statistic=0.81132768286727652, pvalue=0.37667821218931918)
hundred thousand F_onewayResult(statistic=0.74646064762372855, pvalue=0.39615212673610622)
hundred ten F_onewayResult(statistic=0.62995257328961762, pvalue=0.43515030889820316)
hundred one F_onewayResu

None of these appear to be very different. I could also normalize the data by dividing by the total gestures the user made, so rather than having a count, each user would have the proportion of their gestures that were a specific gesture. 

In [167]:
data

[{'box': 0,
  'condition': 'unknown',
  'doubletap': 0,
  'drag': 19,
  'draw': 3,
  'hold': 1,
  'lasso': 0,
  'multi': False,
  'other': 1,
  'pinch': 1,
  'rev_pinch': 0,
  'tap': 0,
  'tripletap': 0,
  'ui': 0,
  'user': u'11',
  'voice': 2},
 {'box': 0,
  'condition': 'unknown',
  'doubletap': 5,
  'drag': 25,
  'draw': 4,
  'hold': 2,
  'lasso': 0,
  'multi': False,
  'other': 1,
  'pinch': 0,
  'rev_pinch': 4,
  'tap': 4,
  'tripletap': 1,
  'ui': 0,
  'user': u'26',
  'voice': 0},
 {'box': 0,
  'condition': 'unknown',
  'doubletap': 0,
  'drag': 13,
  'draw': 3,
  'hold': 2,
  'lasso': 1,
  'multi': False,
  'other': 1,
  'pinch': 1,
  'rev_pinch': 0,
  'tap': 2,
  'tripletap': 0,
  'ui': 0,
  'user': u'21',
  'voice': 0},
 {'box': 0,
  'condition': 'unknown',
  'doubletap': 0,
  'drag': 34,
  'draw': 1,
  'hold': 0,
  'lasso': 0,
  'multi': False,
  'other': 0,
  'pinch': 0,
  'rev_pinch': 0,
  'tap': 0,
  'tripletap': 0,
  'ui': 0,
  'user': u'16',
  'voice': 0},
 {'box': 0,


In [169]:
normalized = []
for entry in data:
    total = 0
    for key in entry.keys():
        if key != "user" and key != "condition":
            total += entry[key]
    for key in entry.keys():
        if key != "user" and key != "condition":
            entry[key] = entry[key]/float(total)
    normalized.append(entry)

In [170]:
normalized

[{'box': 0.0,
  'condition': 'unknown',
  'doubletap': 0.0,
  'drag': 0.7037037037037037,
  'draw': 0.1111111111111111,
  'hold': 0.037037037037037035,
  'lasso': 0.0,
  'multi': 0.0,
  'other': 0.037037037037037035,
  'pinch': 0.037037037037037035,
  'rev_pinch': 0.0,
  'tap': 0.0,
  'tripletap': 0.0,
  'ui': 0.0,
  'user': u'11',
  'voice': 0.07407407407407407},
 {'box': 0.0,
  'condition': 'unknown',
  'doubletap': 0.10869565217391304,
  'drag': 0.5434782608695652,
  'draw': 0.08695652173913043,
  'hold': 0.043478260869565216,
  'lasso': 0.0,
  'multi': 0.0,
  'other': 0.021739130434782608,
  'pinch': 0.0,
  'rev_pinch': 0.08695652173913043,
  'tap': 0.08695652173913043,
  'tripletap': 0.021739130434782608,
  'ui': 0.0,
  'user': u'26',
  'voice': 0.0},
 {'box': 0.0,
  'condition': 'unknown',
  'doubletap': 0.0,
  'drag': 0.5652173913043478,
  'draw': 0.13043478260869565,
  'hold': 0.08695652173913043,
  'lasso': 0.043478260869565216,
  'multi': 0.0,
  'other': 0.043478260869565216,

In [171]:
normf = pandas.DataFrame(normalized)

In [172]:
normf.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,0.000000,unknown,0.000000,0.703704,0.111111,0.037037,0.000000,0.000000,0.037037,0.037037,0.000000,0.000000,0.000000,0.000000,0.074074
26,0.000000,unknown,0.108696,0.543478,0.086957,0.043478,0.000000,0.000000,0.021739,0.000000,0.086957,0.086957,0.021739,0.000000,0.000000
21,0.000000,unknown,0.000000,0.565217,0.130435,0.086957,0.043478,0.000000,0.043478,0.043478,0.000000,0.086957,0.000000,0.000000,0.000000
16,0.000000,unknown,0.000000,0.971429,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,unknown,0.140000,0.560000,0.100000,0.100000,0.000000,0.000000,0.000000,0.020000,0.040000,0.040000,0.000000,0.000000,0.000000
6,0.000000,unknown,0.000000,0.263158,0.236842,0.000000,0.000000,0.000000,0.026316,0.013158,0.039474,0.421053,0.000000,0.000000,0.000000
25,0.000000,thousand,0.000000,0.531250,0.156250,0.000000,0.000000,0.031250,0.000000,0.031250,0.000000,0.000000,0.000000,0.000000,0.250000
15,0.000000,thousand,0.000000,0.500000,0.291667,0.000000,0.138889,0.013889,0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.027778
10,0.000000,thousand,0.000000,0.418605,0.023256,0.000000,0.000000,0.023256,0.488372,0.000000,0.000000,0.023256,0.000000,0.000000,0.023256


In [173]:
model = ols('lasso ~ condition', data=normf).fit()
table = sm.stats.anova_lm(model, typ=1)

In [174]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,0.052829,0.013207,1.22829,0.324124
Residual,25,0.268812,0.010752,NaN,NaN


In [179]:
model = ols('lasso ~ multi', data=normf).fit()
table = sm.stats.anova_lm(model, typ=1)

In [180]:
table

,df,sum_sq,mean_sq,F,PR(>F)
multi,1,0.000558,0.000558,0.048642,0.827043
Residual,28,0.321083,0.011467,NaN,NaN


In [182]:
normf_means = normf.groupby('condition').mean()

In [184]:
normf_means

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
condition,,,,,,,,,,,,,,
hundred,0.080526,0.010365,0.362084,0.097192,0.022059,0.106059,0.017192,0.042642,0.000000,0.005540,0.095451,0.000000,0.033348,0.127542
one,0.000000,0.030483,0.579569,0.088119,0.030819,0.033069,0.000000,0.000000,0.000000,0.004274,0.120350,0.000000,0.080163,0.033154
ten,0.038388,0.024198,0.446563,0.023526,0.020602,0.109177,0.008424,0.013588,0.026726,0.000000,0.186617,0.015587,0.085550,0.001055
thousand,0.000000,0.000000,0.547157,0.107675,0.004386,0.031925,0.024562,0.152017,0.009594,0.027228,0.025508,0.000000,0.019774,0.050172
unknown,0.000000,0.041449,0.601164,0.115653,0.044579,0.007246,0.000000,0.021428,0.018946,0.027738,0.105828,0.003623,0.000000,0.012346


In [191]:
normf_means.loc['hundred'].values

array([ 0.08052608,  0.01036535,  0.36208384,  0.09719229,  0.02205854,
        0.10605869,  0.01719168,  0.04264237,  0.        ,  0.00554029,
        0.09545131,  0.        ,  0.03334754,  0.127542  ])

In [192]:
stats.f_oneway(normf_means.loc['unknown'].values, normf_means.loc['one'].values, normf_means.loc['ten'].values, normf_means.loc['hundred'].values, normf_means.loc['thousand'].values)

F_onewayResult(statistic=-1.3111312516875899e-32, pvalue=nan)

In [193]:
stats.f_oneway(normf_means.loc['unknown'].values, normf_means.loc['one'].values)

F_onewayResult(statistic=5.5117432912327572e-32, pvalue=1.0)

In [194]:
stats.f_oneway(normf_means.loc['unknown'].values, normf_means.loc['hundred'].values)

F_onewayResult(statistic=1.1163437342180082e-31, pvalue=1.0)

This doesn't appear to be useful either, as this is saying that the population means are identical (or NaN for p values, which seems even less useful). This is probably because the normalization means that the mean for a user should be 1.0, so the mean for a population should be very close to 1.0 as well, and so now everything has a population mean of 1.0, and so of course they're not different. 

In [195]:
nonnorm_means = df.groupby('condition').mean()

In [196]:
nonnorm_means

,box,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
condition,,,,,,,,,,,,,,
hundred,6.333333,0.666667,26.833333,9.166667,1.500000,9.166667,True,3.000000,0.000000,0.333333,10.333333,0.000000,3.166667,3.833333
one,0.000000,0.833333,12.166667,2.000000,0.833333,0.500000,False,0.000000,0.000000,0.166667,3.666667,0.000000,2.666667,0.833333
ten,4.333333,2.666667,58.000000,2.333333,2.500000,10.500000,True,1.666667,3.000000,0.000000,29.333333,1.500000,10.000000,0.166667
thousand,0.000000,0.000000,22.666667,5.666667,0.166667,2.000000,True,7.666667,0.333333,1.666667,1.166667,0.000000,1.166667,1.833333
unknown,0.000000,2.000000,23.166667,5.666667,1.666667,0.166667,False,0.833333,0.666667,1.500000,6.666667,0.166667,0.000000,0.333333


In [197]:
stats.f_oneway(nonnorm_means.loc['unknown'].values, nonnorm_means.loc['one'].values, nonnorm_means.loc['ten'].values, nonnorm_means.loc['hundred'].values, nonnorm_means.loc['thousand'].values)

F_onewayResult(statistic=1.4819970777166362, pvalue=0.21788034824896824)

In [200]:
for x in nonnorm_means.index:
    for y in nonnorm_means.index:
        if x != y:
            print x, y, stats.f_oneway(nonnorm_means.loc[x], nonnorm_means.loc[y])
    print

hundred one F_onewayResult(statistic=3.0904602170966049, pvalue=0.090521440218335927)
hundred ten F_onewayResult(statistic=0.61932135902693231, pvalue=0.43841466269354512)
hundred thousand F_onewayResult(statistic=0.73372121381040278, pvalue=0.39950587271930893)
hundred unknown F_onewayResult(statistic=0.84576190859115785, pvalue=0.36620816278713753)

one hundred F_onewayResult(statistic=3.0904602170966058, pvalue=0.090521440218335927)
one ten F_onewayResult(statistic=2.8572082013516469, pvalue=0.10292165478520465)
one thousand F_onewayResult(statistic=0.71836825446650476, pvalue=0.40441469907390104)
one unknown F_onewayResult(statistic=0.54364495881357811, pvalue=0.4675283695418242)

ten hundred F_onewayResult(statistic=0.61932135902693231, pvalue=0.43841466269354512)
ten one F_onewayResult(statistic=2.857208201351646, pvalue=0.10292165478520465)
ten thousand F_onewayResult(statistic=1.6266971580095018, pvalue=0.21343907688359204)
ten unknown F_onewayResult(statistic=1.718489287578483

hundred one F_onewayResult(statistic=3.0445401036633575, pvalue=0.093803195615485752)  
hundred ten F_onewayResult(statistic=0.62995257328961762, pvalue=0.43515030889820316)  
hundred thousand F_onewayResult(statistic=0.74646064762372855, pvalue=0.39615212673610622)  
hundred unknown F_onewayResult(statistic=0.81132768286727652, pvalue=0.37667821218931918)  

one hundred F_onewayResult(statistic=3.0445401036633579, pvalue=0.093803195615485752)  
one ten F_onewayResult(statistic=2.8456501473790681, pvalue=0.10457865347925255)
one thousand F_onewayResult(statistic=0.6589196563127182, pvalue=0.42491972670585898)  
one unknown F_onewayResult(statistic=0.55200384002671354, pvalue=0.46471106985119826)  

ten hundred F_onewayResult(statistic=0.62995257328961785, pvalue=0.43515030889820316)  
ten one F_onewayResult(statistic=2.8456501473790681, pvalue=0.10457865347925255)  
ten thousand F_onewayResult(statistic=1.6498127555285791, pvalue=0.21124481628590228)  
ten unknown F_onewayResult(statistic=1.7036814881039264, pvalue=0.20418288297121795)  

thousand hundred F_onewayResult(statistic=0.74646064762372843, pvalue=0.39615212673610622)  
thousand one F_onewayResult(statistic=0.6589196563127182, pvalue=0.42491972670585898)  
thousand ten F_onewayResult(statistic=1.6498127555285791, pvalue=0.21124481628590228)  
thousand unknown F_onewayResult(statistic=0.002188696239585675, pvalue=0.96307289865537826)  

unknown hundred F_onewayResult(statistic=0.81132768286727719, pvalue=0.37667821218931918)  
unknown one F_onewayResult(statistic=0.55200384002671332, pvalue=0.46471106985119826)  
unknown ten F_onewayResult(statistic=1.7036814881039262, pvalue=0.20418288297121795)  
unknown thousand F_onewayResult(statistic=0.0021886962395856746, pvalue=0.96307289865537826)  

This is the values from the averages that I calculated with python, it seems pretty close to what pandas came up with (although not identical).  

In [203]:
groups = df.groupby('condition').groups

In [219]:
tens = df.loc[groups['ten']].drop(['condition', 'multi'],axis=1)

In [220]:
unknowns = df.loc[groups['unknown']].drop(['condition', 'multi'],axis=1)

In [224]:
thousands = df.loc[groups['thousand']].drop(['condition', 'multi'],axis=1)

In [223]:
ones = df.loc[groups['one']].drop(['condition', 'multi'],axis=1)

In [225]:
hundreds = df.loc[groups['hundred']].drop(['condition', 'multi'],axis=1)

In [226]:
stats.f_oneway(unknowns, ones, tens, hundreds, thousands)

F_onewayResult(statistic=array([ 1.38171312,  1.62015504,  3.98631722,  1.06225093,  1.82291667,
        1.78755227,  1.56672378,  1.67056075,  1.34661836,  1.7763003 ,
        1.99742268,  0.8334859 ,  0.17142857,  0.96099624]), pvalue=array([ 0.26875503,  0.20034268,  0.01230788,  0.39579935,  0.15594663,
        0.16290879,  0.21400734,  0.18824629,  0.28056786,  0.16518902,
        0.12575949,  0.51671899,  0.95096385,  0.44606218]))

Again, this makes it look like there isn't a statistically significant variation in the data on each class. 

In [233]:
stats.f_oneway(unknowns['lasso'].values, ones['lasso'].values, tens['lasso'].values, hundreds['lasso'].values, thousands['lasso'].values)

F_onewayResult(statistic=1.7875522676101636, pvalue=0.16290878513070095)

In [234]:
groups = df.groupby('multi').groups

In [237]:
multis = df.loc[groups[True]].drop(['condition','multi'], axis = 1)
singles = df.loc[groups[False]].drop(['condition','multi'], axis = 1)

In [238]:
stats.f_oneway(multis, singles)

F_onewayResult(statistic=array([ 2.30343442,  0.13974015,  4.12045044,  0.51625663,  0.04712218,
        4.11712728,  2.76074803,  0.69118388,  0.06693227,  1.0957228 ,
        0.84507042,  0.79468399,  0.57207719,  0.94348467]), pvalue=array([ 0.14029922,  0.71135521,  0.05196674,  0.47839626,  0.82972278,
        0.05205525,  0.10776081,  0.41279597,  0.79774933,  0.30415923,
        0.36580268,  0.3802844 ,  0.45575179,  0.33970001]))

In [239]:
stats.f_oneway(multis['lasso'].values, singles['lasso'].values)

F_onewayResult(statistic=4.1171272831596069, pvalue=0.052055249058089066)

In [243]:
df.groupby('condition').describe()

box  doubletap        drag       draw      hold  \
condition                                                                
hundred   count   6.000000   6.000000    6.000000   6.000000  6.000000   
          mean    6.333333   0.666667   26.833333   9.166667  1.500000   
          std    10.132456   1.211060   22.382285  11.178849  1.760682   
          min     0.000000   0.000000    5.000000   0.000000  0.000000   
          25%     0.000000   0.000000    9.000000   2.000000  0.000000   
          50%     0.000000   0.000000   21.000000   6.000000  1.000000   
          75%    11.250000   0.750000   43.500000  10.750000  2.750000   
          max    23.000000   3.000000   58.000000  30.000000  4.000000   
one       count   6.000000   6.000000    6.000000   6.000000  6.000000   
          mean    0.000000   0.833333   12.166667   2.000000  0.833333   
          std     0.000000   1.169045    2.857738   2.280351  1.169045   
          min     0.000000   0.000000   10.000000   0.000000  0.000000   
          25%     0.000000   0.000000   10.000000   0.250000  0.000000   
          50%     0.000000   0.500000   11.000000   1.500000  0.500000   
          75%     0.000000   1.000000   13.500000   2.750000  1.000000   
          max     0.000000   3.000000   17.000000   6.000000  3.000000   
ten       count   6.000000   6.000000    6.000000   6.000000  6.000000   
          mean    4.333333   2.666667   58.000000   2.333333  2.500000   
          std     9.667816   2.943920   40.274061   3.881580  2.167948   
          min     0.000000   0.000000    7.000000   0.000000  0.000000   
          25%     0.000000   1.000000   31.250000   0.000000  0.500000   
          50%     0.000000   1.500000   54.500000   1.000000  3.000000   
          75%     1.500000   3.500000   86.000000   2.000000  4.000000   
          max    24.000000   8.000000  112.000000  10.000000  5.000000   
thousand  count   6.000000   6.000000    6.000000   6.000000  6.000000   
          mean    0.000000   0.000000   22.666667   5.666667  0.166667   
          std     0.000000   0.000000    8.710147   7.685484  0.408248   
          min     0.000000   0.000000   12.000000   1.000000  0.000000   
          25%     0.000000   0.000000   17.250000   1.250000  0.000000   
          50%     0.000000   0.000000   21.500000   3.000000  0.000000   
          75%     0.000000   0.000000   27.250000   4.750000  0.000000   
          max     0.000000   0.000000   36.000000  21.000000  1.000000   
unknown   count   6.000000   6.000000    6.000000   6.000000  6.000000   
          mean    0.000000   2.000000   23.166667   5.666667  1.666667   
          std     0.000000   3.162278    7.413951   6.186006  1.861899   
          min     0.000000   0.000000   13.000000   1.000000  0.000000   
          25%     0.000000   0.000000   19.250000   3.000000  0.250000   
          50%     0.000000   0.000000   22.500000   3.500000  1.500000   
          75%     0.000000   3.750000   27.250000   4.750000  2.000000   
          max     0.000000   7.000000   34.000000  18.000000  5.000000   

                     lasso  multi      other      pinch  rev_pinch  \
condition                                                            
hundred   count   6.000000      6   6.000000   6.000000   6.000000   
          mean    9.166667      1   3.000000   0.000000   0.333333   
          std    11.651895      0   5.477226   0.000000   0.516398   
          min     0.000000   True   0.000000   0.000000   0.000000   
          25%     0.500000      1   0.000000   0.000000   0.000000   
          50%     4.500000      1   1.000000   0.000000   0.000000   
          75%    14.500000      1   2.000000   0.000000   0.750000   
          max    29.000000   True  14.000000   0.000000   1.000000   
one       count   6.000000      6   6.000000   6.000000   6.000000   
          mean    0.500000      0   0.000000   0.000000   0.166667   
          std     0.836660      0   0.000000   0.000000   0.408248   
    

The standard deviations are all over the place, ANOVA expects groups to have similar standard deviations. There is probably some normalization method to account for this. 

In [249]:
for col in df.columns:
    if col != 'condition' and col != 'multi':
        print col, stats.shapiro(df[col])

 box (0.37173521518707275, 2.9864827277847894e-10)
doubletap (0.644141674041748, 2.640740888182336e-07)
drag (0.7698088884353638, 1.93547075468814e-05)
draw (0.6934834718704224, 1.2559264632727718e-06)
hold (0.7784423828125, 2.718419455050025e-05)
lasso (0.5368094444274902, 1.34227793324726e-08)
other (0.47543811798095703, 2.9754634311984773e-09)
pinch (0.35434842109680176, 2.0841951775540934e-10)
rev_pinch (0.507911741733551, 6.5027285778285204e-09)
tap (0.5210667848587036, 9.012343760161912e-09)
tripletap (0.3125055432319641, 9.004358370034993e-11)
ui (0.36672455072402954, 2.69053862522739e-10)
user (0.0019217133522033691, 4.3953320258589834e-13)
voice (0.429931104183197, 1.048225839461736e-09)


The _really tiny_ p values on all of these seems to indicate that my data is sampled from a normal distribution, which is good for ANOVA. I probably only have to worry about fixing the standard deviations, not normalizing the distribution. 

In [267]:
groups = df.groupby('condition').groups
for group in groups:
    g = df.loc[groups[group]]
    print group
    for col in g.columns:
        if col != 'condition' and col != 'multi':
            print col, stats.shapiro(g[col])

unknown
box (1.0, 1.0)
doubletap (0.6972294449806213, 0.005778977647423744)
drag (0.9870100021362305, 0.9806381464004517)
draw (0.7008750438690186, 0.006314219906926155)
hold (0.8616126775741577, 0.1947678178548813)
lasso (0.49609434604644775, 2.0729139578179456e-05)
other (0.8662614226341248, 0.2117050290107727)
pinch (0.6398939490318298, 0.0013507615076377988)
rev_pinch (0.8317347764968872, 0.11118293553590775)
tap (0.6025598645210266, 0.0004925247048959136)
tripletap (0.49609434604644775, 2.0729139578179456e-05)
ui (1.0, 1.0)
user (0.14145517349243164, 7.39237924346492e-12)
voice (0.49609434604644775, 2.0729139578179456e-05)
thousand
box (1.0, 1.0)
doubletap (1.0, 1.0)
drag (0.9654346108436584, 0.8604826331138611)
draw (0.6787958145141602, 0.0036668768152594566)
hold (0.49609434604644775, 2.0729139578179456e-05)
lasso (0.5925898551940918, 0.00037298371898941696)
other (0.6766088008880615, 0.003471522592008114)
pinch (0.6398937106132507, 0.0013507531257346272)
rev_pinch (0.6340762376

/usr/lib/python2.7/dist-packages/scipy/stats/morestats.py:1326: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


The normality test mostly stays small when checking within groups, but not for all groups

In [313]:
groups = df.groupby('condition').groups
znorm = []
for group in groups:
    g = df.loc[groups[group]]
    means = g.mean(numeric_only=True)
    std_devs = g.std(numeric_only=True)
    #print means['box'], std_devs['box']
    #For each row in the group
    gdata = []
    for row in g.iterrows():
        #For each value in the row, calculate its zscore
        zscores = {}
        for index in df.axes[1].tolist():
            if index != 'condition' and index != "multi" and index != "user":
                if std_devs[index] != 0:
                    zscores[index] = (row[1][index] - means[index])/std_devs[index]   
                else: 
                    zscores[index] = 0.0
        #Put the user ID back in
        zscores['user'] = row[1]['user']
        zscores['condition'] = row[1]['condition']
        znorm.append(zscores)
        
    

This is an attempt to z-score the data within each gesture and within each group, so taking the mean and standard deviation of (for example) "drag" in the 10-robot case, and then using those values to z-score each user in the 10-robot case. 

It would also be possible to take the mean and std. dev. of all of a user's gestures, and then z-score all of their gestures, so that "gestures used" would have a mean of 0 and std. dev. of 1. I'd expect that to have a similar result to the attempt to normalize gestures by dividing by total gesture count per user, where the counts would get turned into a proportion.

In [314]:
znorm_df = pandas.DataFrame(znorm)

In [316]:
znorm_df.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,
11,0.000000,unknown,-0.632456,-0.562004,-0.431081,-0.358057,-0.408248,0.221404,0.645497,-0.851943,-0.533305,-0.408248,0.000000,2.041241
26,0.000000,unknown,0.948683,0.247282,-0.269425,0.179029,-0.408248,0.221404,-1.290994,1.419905,-0.213322,2.041241,0.000000,-0.408248
21,0.000000,unknown,-0.632456,-1.371289,-0.431081,0.179029,2.041241,0.221404,0.645497,-0.851943,-0.373313,-0.408248,0.000000,-0.408248
16,0.000000,unknown,-0.632456,1.461209,-0.754391,-0.895144,-0.408248,-1.107019,-1.290994,-0.851943,-0.533305,-0.408248,0.000000,-0.408248
1,0.000000,unknown,1.581139,0.651924,-0.107770,1.790287,-0.408248,-1.107019,0.645497,0.283981,-0.373313,-0.408248,0.000000,-0.408248
6,0.000000,unknown,-0.632456,-0.427123,1.993747,-0.895144,-0.408248,1.549826,0.645497,0.851943,2.026559,-0.408248,0.000000,-0.408248
25,0.000000,thousand,0.000000,-0.650582,-0.086744,-0.408248,-0.506370,-0.641867,1.290994,-0.520156,-0.792594,0.000000,-0.408248,1.973228
15,0.000000,thousand,0.000000,1.530782,1.995103,-0.408248,2.025479,-0.641867,-0.645497,0.104031,-0.792594,0.000000,-0.408248,0.053330
10,0.000000,thousand,0.000000,-0.535774,-0.607205,-0.408248,-0.506370,1.116291,-0.645497,-0.520156,-0.113228,0.000000,-0.408248,-0.266652


In [317]:
model = ols('lasso ~ condition', data=znorm_df).fit()
table = sm.stats.anova_lm(model, typ=1)

In [318]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,1.548448e-31,3.871119e-32,3.871119e-32,1
Residual,25,2.500000e+01,1.000000e+00,NaN,NaN


In [319]:
groups = znorm_df.groupby('condition').groups

In [320]:
unknowns = znorm_df.loc[groups['unknown']].drop(['condition'],axis=1)

In [321]:
tens = znorm_df.loc[groups['ten']].drop(['condition'],axis=1)

In [322]:
stats.f_oneway(unknowns, tens)

F_onewayResult(statistic=array([ -1.06581410e-14,   0.00000000e+00,  -1.77635684e-15,
        -1.77635684e-15,  -1.77635684e-15,  -1.77635684e-15,
         8.88178420e-15,  -5.32907052e-15,  -1.77635684e-14,
        -1.77635684e-15,   0.00000000e+00,   7.10542736e-15,
         1.37142857e-01,   5.32907052e-15]), pvalue=array([        nan,  1.        ,         nan,         nan,         nan,
               nan,  0.99999993,         nan,         nan,         nan,
        1.        ,  0.99999994,  0.71886143,  0.99999994]))

Clearly, Z-scores were not the way to go here. What I would want to see is lower p-values, these are nearly one. These are normalized with the means and standard deviations of COLUMNS, because I want to be able to compare the same gestures across users, and normalizing across all the gestures that a user did seems like the wrong thing to do. 

In [323]:
stats.f_oneway(unknowns['lasso'].values, tens['lasso'].values)

F_onewayResult(statistic=6.162975822039154e-33, pvalue=1.0)

In [335]:
df.set_index('user')
row_stddevs = df.std(axis=1, numeric_only=True)
row_means = df.mean(axis=1, numeric_only=True)

In [347]:
rownormed = []
for row in df.iterrows():
    rowdata = {}
    for index in df.axes[1].tolist():
        if index == 'user' or index == 'condition' or index == 'multi':
            rowdata[index] = row[1][index]
        else:
            if row_stddevs[row[0]] == 0:
                rowdata[index] = 0
            else:
                rowdata[index] = (row[1][index]-row_means[row[0]])/row_stddevs[row[0]]
    rownormed.append(rowdata)

In [348]:
rownormed_df = pandas.DataFrame(rownormed)
rownormed_df.set_index('user')

,box,condition,doubletap,drag,draw,hold,lasso,multi,other,pinch,rev_pinch,tap,tripletap,ui,voice
user,,,,,,,,,,,,,,,
11,-0.385757,unknown,-0.385757,3.414661,0.214309,-0.185735,-0.385757,False,-0.185735,-0.185735,-0.385757,-0.385757,-0.385757,-0.385757,0.014287
26,-0.503843,unknown,0.262874,3.329743,0.109531,-0.197156,-0.503843,False,-0.350499,-0.503843,0.109531,0.109531,-0.350499,-0.503843,-0.503843
21,-0.481698,unknown,-0.481698,3.330001,0.397925,0.104717,-0.188491,False,-0.188491,-0.188491,-0.481698,0.104717,-0.481698,-0.481698,-0.481698
16,-0.275627,unknown,-0.275627,3.472896,-0.165376,-0.275627,-0.275627,False,-0.275627,-0.275627,-0.275627,-0.275627,-0.275627,-0.275627,-0.275627
1,-0.482101,unknown,0.462817,3.297574,0.192841,0.192841,-0.482101,False,-0.482101,-0.347113,-0.212125,-0.212125,-0.482101,-0.482101,-0.482101
6,-0.532822,unknown,-0.532822,1.430207,1.233904,-0.532822,-0.532822,False,-0.336519,-0.434671,-0.238368,2.608024,-0.532822,-0.532822,-0.532822
25,-0.470463,thousand,-0.470463,3.028606,0.558675,-0.470463,-0.470463,True,-0.470463,-0.264635,-0.470463,-0.470463,-0.470463,-0.470463,1.176158
15,-0.482697,thousand,-0.482697,2.896185,1.488317,-0.482697,0.455881,True,-0.482697,-0.482697,-0.294982,-0.482697,-0.482697,-0.482697,-0.294982
10,-0.438800,thousand,-0.438800,2.132773,-0.295935,-0.438800,-0.438800,True,2.561368,-0.438800,-0.438800,-0.295935,-0.438800,-0.438800,-0.295935


In [345]:
model = ols('lasso ~ condition', data=znorm_df).fit()
table = sm.stats.anova_lm(model, typ=1)

In [346]:
table

,df,sum_sq,mean_sq,F,PR(>F)
condition,4,1.548448e-31,3.871119e-32,3.871119e-32,1
Residual,25,2.500000e+01,1.000000e+00,NaN,NaN
